In [ ]:
# Imports here
from torchvision import datasets, transforms, models
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                     transforms.RandomResizedCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
# transforms.Normalize((0.5,0.5), (0.5,0.5)),

train = datasets.ImageFolder(train_dir,transform=train_transforms)
valid = datasets.ImageFolder(valid_dir,transform=test_transforms)
test = datasets.ImageFolder(test_dir,transform=test_transforms)

train_load = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)
valid_load = torch.utils.data.DataLoader(valid, batch_size=64)
test_load = torch.utils.data.DataLoader(test, batch_size=64)

In [ ]:
train

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
print(len(cat_to_name))

In [ ]:
import torchvision.models as models


vgg16 = models.vgg16(pretrained=True)

hidden_size = 512
output_size=102
input_size=25088
#224*224
for param in vgg16.parameters():
    param.requires_grad = False
    
vgg16.classifier=nn.Sequential(OrderedDict([
                      ('fc1', nn.Linear(input_size, hidden_size)),
                      ('drop', nn.Dropout(p = 0.5)),
                      ('relu1', nn.ReLU()),
                      ('output', nn.Linear(hidden_size, output_size)),
                      ('softmax', nn.LogSoftmax(dim=1))]))
#
criterion=nn.NLLLoss()
LR=0.001
optimizer = optim.Adam(vgg16.classifier.parameters(), lr = LR)

epochs=3
vgg16.class_to_idx = train.class_to_idx
batch_size=64
vgg16.cuda()
for e in range(epochs):
    running_loss = 0
    print(e)
    i=1
    for images, labels in train_load:
        images=images.cuda()
        labels=labels.cuda()
        #= images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = vgg16.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print("Still running",round(64*i/len(train),2)*100,"%")
        i+=1
        running_loss += loss.item()
        #if i==10:
        #    break
    valid_loss=0
    acc=0
    vgg16.eval()
    i=1
    with torch.no_grad():
        for images, labels in valid_load:
            images=images.cuda()
            labels=labels.cuda()
            #images, labels = images.to(device), labels.to(device)
            output=vgg16.forward(images)
            loss=criterion(output,labels)
            valid_loss+=loss.item()
            ps= torch.exp(output)
            top_p, top_class = ps.topk(1, dim = 1)
            equals = top_class == labels.view(*top_class.shape)
            acc+= torch.mean(equals.type(torch.FloatTensor)).item()
            #if i==10:
            #    break
    vgg16.train()
    print(f"Training loss: {running_loss/len(train_load)}")
    print(f"Accuracy: {acc / len(valid_load)}")

In [ ]:
valid_loss=0
acc=0
vgg16.eval()
i=1
with torch.no_grad():
    for images, labels in test_load:
        images=images.cuda()
        labels=labels.cuda()
        #images, labels = images.to(device), labels.to(device)
        output=vgg16.forward(images)
        loss=criterion(output,labels)
        valid_loss+=loss.item()
        ps= torch.exp(output)
        top_p, top_class = ps.topk(1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        acc+= torch.mean(equals.type(torch.FloatTensor)).item()
        #if i==10:
        #    break
vgg16.train()
print(f"Training loss: {running_loss/len(train_load)}")
print(f"Accuracy: {acc / len(valid_load)}")

In [ ]:
vgg16.class_to_idx = train.class_to_idx
batch_size=64
checkpoint = {'network': 'vgg16',
              'input_size': input_size,
              'output_size': output_size,
              'learning_rate': LR,       
              'batch_size': batch_size,
              'classifier' :vgg16.classifier,
              'epochs': epochs,
              'optimizer': optimizer.state_dict(),
              'state_dict': vgg16.state_dict(),
              'class_to_idx': vgg16.class_to_idx}

torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
def load_checkpoint(file_path):
    checkpoint = torch.load(file_path)
    LR = checkpoint['learning_rate']
    model = getattr(models, checkpoint['network'])(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.epochs = checkpoint['epochs']
    model.optimizer = checkpoint['optimizer']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    
    return model

vgg16 = load_checkpoint('checkpoint.pth')
print(vgg16)

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    i_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(244), 
                                     transforms.ToTensor(),])
    np_image = np.array(i_transform(image).float())

    np_image=(np.transpose(np_image,(1,2,0))-np.array([0.485,0.456,0.406]))/np.array([0.229,0.224,0.225])
    np_image = np.transpose(np_image, (2, 0, 1))
    return(np_image)

import PIL
import os
import numpy as np
ldir=os.listdir(train_dir+"/1")
image= PIL.Image.open(train_dir+"/1/image_06735.jpg")
plt.imshow(image)

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    #image = image.numpy().transpose((1, 2, 0))
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

imshow(process_image(image))

In [ ]:
from torch.autograd import Variable
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.cuda()
    model.eval()
    image = PIL.Image.open(image_path)
    np_image=process_image(image)
    tensor_image=torch.from_numpy(np_image).unsqueeze_(0).float().cuda()
    logp = model.forward(tensor_image)
    prob = torch.exp(logp)    
    top_p, top_classes = prob.topk(topk, dim = 1)
    idx = {model.class_to_idx[c]: c for c in model.class_to_idx}
    top_c=[cat_to_name[idx[l]] for l in  top_classes.cpu().detach().numpy()[0]]

    return(top_p.cpu().detach().numpy()[0],top_c)
im_path=train_dir+"/1/image_06735.jpg"
top_p,top_c=predict(im_path,vgg16)
print(top_p)
print(top_c)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax1 = plt.subplot2grid((15,9), (0,0), colspan = 9, rowspan = 9)
ax2 = plt.subplot2grid((15,9), (9,2), colspan = 5, rowspan = 5)

ax1.axis('off')
ax1.set_title(top_c[np.argmax(top_p)])
ax1.imshow(image)


y_pos = np.arange(5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(top_c)
ax2.set_xlabel('Probability')
ax2.invert_yaxis()
ax2.barh(y_pos, top_p, xerr = 0, align = 'center', color = 'blue')

plt.show()

In [ ]:
for i in range(10):
    ldir=os.listdir(train_dir+"/"+str(i+1))
    im_path=train_dir+"/"+str(i+1)+"/"+ldir[i]
    print(im_path)
    top_p,top_c=predict(im_path,vgg16)
    fig = plt.figure(figsize=(6,6))
    ax1 = plt.subplot2grid((15,9), (0,0), colspan = 9, rowspan = 9)
    ax2 = plt.subplot2grid((15,9), (9,2), colspan = 5, rowspan = 5)

    ax1.axis('off')
    ax1.set_title(top_c[np.argmax(top_p)])
    image = PIL.Image.open(im_path)
    ax1.imshow(image)


    y_pos = np.arange(5)
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(top_c)
    ax2.set_xlabel('Probability')
    ax2.invert_yaxis()
    ax2.barh(y_pos, top_p, xerr = 0, align = 'center', color = 'blue')

    plt.show()